In [1]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
info = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(f'Total memory: {info.total / 1024**3:.2f} GB')
print(f'Free memory: {info.free / 1024**3:.2f} GB')
print(f'Used memory: {info.used / 1024**3:.2f} GB')
import dask_cudf
import cupy as cp
import rmm
import cudf
import gc
import cudf.pandas
cudf.pandas.install()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Total memory: 8.00 GB
Free memory: 7.60 GB
Used memory: 0.40 GB


In [2]:
subway_surfers = dask_cudf.read_csv('MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240531.csv', blocksize="10MB") 
subway_pd = pd.read_csv('MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240531.csv')

/home/eamonn-walsh/miniconda3/envs/rapids-24.04/lib/python3.11/site-packages/cudf/pandas/fast_slow_proxy.py:30: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return fn(*args, **kwargs)


In [3]:
subway_surfers.columns

Index(['transit_timestamp', 'transit_mode', 'station_complex_id',
       'station_complex', 'borough', 'payment_method', 'fare_class_category',
       'ridership', 'transfers', 'latitude', 'longitude', 'Georeference'],
      dtype='object')

In [4]:
subway_surfers = subway_surfers[["transit_timestamp","transit_mode","borough","ridership","latitude","longitude","Georeference"]]

In [5]:
subway_pd = subway_pd[["transit_timestamp","transit_mode","borough","ridership","latitude","longitude","Georeference"]]


In [6]:
dtypes = subway_surfers.dtypes
print(dtypes)
# subway_surfers['station_complex_id'] = subway_surfers['station_complex_id'].astype('int64')

transit_timestamp     object
transit_mode          object
borough               object
ridership              int64
latitude             float64
longitude            float64
Georeference          object
dtype: object


In [7]:
rows = subway_surfers.shape[0].compute()
print(rows)

58306773


In [8]:
null_counts = subway_surfers.isnull().sum()
print(null_counts.compute())

transit_timestamp    0
transit_mode         0
borough              0
ridership            0
latitude             0
longitude            0
Georeference         0
dtype: int64


In [9]:
print(type(subway_surfers))

<class 'dask_cudf.core.DataFrame'>


In [10]:
for col in subway_surfers:
    print(subway_surfers.groupby(col).size())

<dask_cudf.Series | 3132 tasks | 1 npartitions>
<dask_cudf.Series | 3132 tasks | 1 npartitions>
<dask_cudf.Series | 3132 tasks | 1 npartitions>
<dask_cudf.Series | 3132 tasks | 1 npartitions>
<dask_cudf.Series | 3132 tasks | 1 npartitions>
<dask_cudf.Series | 3132 tasks | 1 npartitions>
<dask_cudf.Series | 3132 tasks | 1 npartitions>


In [11]:
subway_surfers.dtypes

transit_timestamp     object
transit_mode          object
borough               object
ridership              int64
latitude             float64
longitude            float64
Georeference          object
dtype: object

In [12]:
lat_lower_bound = 40.492
lat_upper_bound = 40.916
long_lower_bound = -74.254
long_upper_bound = -73.698

filtered_data = subway_surfers[
    (subway_surfers['latitude'] < lat_lower_bound) |
    (subway_surfers['latitude'] > lat_upper_bound) |
    (subway_surfers['longitude'] < long_lower_bound) |
    (subway_surfers['longitude'] > long_upper_bound)
]

filtered_data_computed = filtered_data.compute()

bound_rows = filtered_data_computed.shape[0]
print(bound_rows)
filtered_data_computed

0


,transit_timestamp,transit_mode,borough,ridership,latitude,longitude,Georeference


In [13]:
subway_pd['transit_timestamp'] = pd.to_datetime(subway_pd['transit_timestamp'])

In [14]:
meta = {
    'transit_timestamp': 'datetime64[ms]',
   
    'transit_mode': 'object',
    'borough': 'object',
    'ridership': 'int16',
    'latitude': 'float32',

    'longitude': 'float32',

    'Georeference': 'object',
   

}
meta_data = subway_surfers.map_partitions(lambda df: df.head(), meta=meta)

In [15]:

pandas_dfs = []
chunk_size = 10  # Adjust chunk size based on your VRAM capacity

for i in range(0, len(meta_data), chunk_size):
    chunk = meta_data.loc[i:i+chunk_size-1].compute().to_pandas()
    pandas_dfs.append(chunk)

# Concatenate all chunks into a single pandas DataFrame
final_pandas_df = pd.concat(pandas_dfs, ignore_index=True)

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/eamonn-walsh/miniconda3/envs/rapids-24.04/include/rmm/mr/device/cuda_memory_resource.hpp

In [ ]:
result  = meta_data.compute()

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/eamonn-walsh/miniconda3/envs/rapids-24.04/include/rmm/mr/device/cuda_memory_resource.hpp

In [16]:
min_date = subway_pd['transit_timestamp'].min()
max_date = subway_pd['transit_timestamp'].max()

# Print the results
print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2022-02-01T00:00:00.000000000
Maximum date: 2024-05-23T23:00:00.000000000


In [18]:
subway_pd.columns

Index(['transit_timestamp', 'transit_mode', 'borough', 'ridership', 'latitude',
       'longitude', 'Georeference'],
      dtype='object')

In [17]:
subway_pd.to_csv('saved_csvs/subway_surfers.csv', index=False)

In [ ]:
subway_pd['transit_timestamp'] = subway_pd['transit_timestamp'].dt.time

# Find the minimum and maximum times
min_time = subway_pd['transit_timestamp'].min()
max_time = subway_pd['transit_timestamp'].max()

# Print the results
print(f"Minimum time: {min_time}")
print(f"Maximum time: {max_time}")

Minimum time: 00:00:00
Maximum time: 23:00:00
